In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, json
import numpy as np

if not os.getcwd().endswith("src"):
    os.chdir("src")
    print("jumping into src")

from utils.data.data_module import DataModule
from utils.data.testbench import TestBench
from recommender.sweep_pipeline import Models

if os.getcwd().endswith("src"):
    os.chdir("..")
    print("jumping out of src")

args = {
    "output_dir": "models/jaccard",
    "dataset_config": "configs/datasets/masked_is_negative.json",
    "model_config": "configs/jaccard/rownormalized.json",
    "model": "jaccard",
    "should_return_ids": False,
}
dataset_config = {}
model_config = {}
output_dir = args["output_dir"]
if args["dataset_config"]:
    with open(args["dataset_config"], "r") as f:
        dataset_config_2 = json.load(f)
        dataset_config.update(dataset_config_2)
if args["model_config"]:
    with open(args["model_config"], "r") as f:
        model_config = json.load(f)

os.makedirs(output_dir, exist_ok=True)
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["TORCH_USE_CUDA_DSA"] = "1"

jumping out of src


In [5]:
datamodule = DataModule(**dataset_config)
testbench = TestBench(
    datamodule,
    should_return_ids=False,
    sigmoid_scores=True,
)

In [6]:
auxiliary_args = {
    "n_users": datamodule.max_user_count,
    "n_anime": datamodule.max_anime_count,
}
model_config = model_config | auxiliary_args
model = Models.from_string(args["model"].upper())(datamodule=datamodule, **model_config)

model.train()

Building interaction matrix...: 100%|██████████| 54077/54077 [00:07<00:00, 7271.87it/s]


In [13]:
metrics = testbench.full_evaluation(model)
with open(os.path.join(output_dir, "output.txt"), "w") as f:
    for k, v in metrics.items():
        if type(v) == np.ndarray:
            continue
        f.write(f"{k}: {v}\n")

Preserved Features: 100%|██████████| 5408/5408 [00:00<00:00, 26220.85it/s]


Start Time: 2024-11-20 13:54:18
End Time: 2024-11-20 13:55:39


c:\Users\James\Desktop\projects\cmu\CMU_10718\src\utils\data\testbench.py:236: RuntimeWarning: invalid value encountered in divide
  (


This model took 81.0610 seconds.
Out of an optimal score of 1.0, you scored 0.2308.
Your DEI score is 9592.2382.
Your Pseudo-IOU score is 0.0163.
Your calibration score is 0.5118.
Your calibration breakdown: 
Bucket Frequency    |Bucket Accuracy     |Bucket Average Probability
     662316.00      |       0.0008       |       0.0000       
        0.00        |       0.0000       |       0.0000       
        0.00        |       0.0000       |       0.0000       
        0.00        |       0.0000       |       0.0000       
        0.00        |       0.0000       |       0.0000       
    62646993.00     |       0.0008       |       0.5100       
     2356905.00     |       0.0008       |       0.6382       
     621225.00      |       0.0008       |       0.7400       
     171915.00      |       0.0008       |       0.8386       
      26598.00      |       0.0008       |       0.9274       
